In [1]:
import ee

import geemap


In [2]:
os.chdir('/Users/najah/work/internships/meghna//LT05_L1TP_145044_20100428_20161016_01_T1')

NameError: name 'os' is not defined

In [ ]:
ee.Authenticate()

In [2]:
print(geemap.__version__)

0.16.7


In [5]:
ee.Initialize()


In [ ]:
Map = geemap.Map()
Map

In [7]:
## adding points from gee
# points = ee.FeatureCollection("projects/fire-ashoka/assets/145044_20100428_500points")

# Map.addLayer(points, {'color':'red'}, '500points')

In [103]:
## adding points from local machine

points_shp = './145044_20100428_roi/145044_20100428_500points.random_points/145044_20100428_500points.shp'
points = geemap.shp_to_ee(points_shp)
Map.addLayer(points, {'color':'red'}, '500points')
Map


Map(bottom=28743.0, center=[23.10324765528739, 78.02415285034276], controls=(WidgetControl(options=['position'…

In [101]:
print(points.first().getInfo())

{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [77.40547254617523, 23.866743087143593]}, 'id': '0', 'properties': {'ag_fire': 0, 'class': 11, 'class1': 11, 'id': 57, 'lat': 23.867, 'long': 77.405}}


In [104]:
image = ee.Image('LANDSAT/LT05/C01/T1_TOA/LT05_145044_20100428') \
            .select(['B1', 'B2', 'B3', 'B4', 'B5', 'B7'])

Map.centerObject(image)

rgbVis = {
  'min':0,
  'max' :0.4,
  'bands' : ['B5', 'B4', 'B3'],
  'gamma' : 1.2
}

Map.addLayer(image, rgbVis, 'original')
Map

Map(bottom=28743.0, center=[23.10324765528739, 78.02415285034276], controls=(WidgetControl(options=['position'…

In [105]:
# Calculate Slope and Elevation
elevation_raster = ee.Image('USGS/SRTMGL1_003')

elev = elevation_raster.select('elevation').rename('elev')
slope = ee.Terrain.slope(elevation_raster.select('elevation')).rename('slope')

image = image.addBands(elev).addBands(slope)

In [ ]:
# hillshadow calc

#ee.Terrain.hillShadow(image, azimuth, zenith, neighborhoodSize, hysteresis)

# def getShadow(image):

#   elevation = image.get('SUN_ELEVATION')
#   zenith = ee.Number(90).subtract(ee.Number(elevation))

#   shadow = ee.Terrain.hillShadow({
#     'image': elevation_raster,
#     'azimuth': image.get('SUN_AZIMUTH'),
#     'zenith': ee.Number(zenith),
#     'neighborhoodSize': 200,
#     'hysteresis': True
#   }).rename('shadow')

#   return image.addBands(shadow)

# image = getShadow(image)

In [106]:
def addIndices(image):

  ndvi = image.normalizedDifference(['B4', 'B3']).rename('ndvi'),

  ndmi = image.normalizedDifference(['B4', 'B5']).rename('ndmi'),

  nbr = image.normalizedDifference(['B4', 'B7']).rename('nbr'),

  ndwi = image.normalizedDifference(['B2', 'B4']).rename('ndwi'),

  bai = image.expression(
    '1/((0.1-red)**2+ (0.06-nir)**2)',{
      'nir': image.select('B4'),
      'red': image.select('B3')

    }).rename('bai')

  return image.addBands(ndvi).addBands(ndmi).addBands(nbr).addBands(ndwi)

In [107]:
image = addIndices(image)

In [108]:
# Use these bands for prediction.
bands = ['B1', 'B2', 'B3', 'B4', 'B5',  'B7', 'ndvi', 'nbr', 'ndwi']

# This property of the table stores the land cover labels.
label = 'class'

# Overlay the points on the imagery to get training.
# sample = image.select(bands).sampleRegions(
#     **{'collection': points, 'properties': [label], 'scale': 30}
# )

sample = image.select(bands).sampleRegions(**{
  'collection' : points,
  'properties' : [label],
  'scale': 30
})

In [114]:
##Adds a column of deterministic pseudorandom numbers.
sample = sample.randomColumn()

split = 0.7

training = sample.filter(ee.Filter.lt('random', split))
validation = sample.filter(ee.Filter.gte('random', split))

training.first().getInfo()

validation.first().getInfo()

{'type': 'Feature',
 'geometry': None,
 'id': '0_0',
 'properties': {'B1': 0.1458006203174591,
  'B2': 0.14518778026103973,
  'B3': 0.17255185544490814,
  'B4': 0.2249782681465149,
  'B5': 0.28518906235694885,
  'B7': 0.20986996591091156,
  'class': 11,
  'nbr': 0.03474384918808937,
  'ndvi': 0.13188035786151886,
  'ndwi': -0.21555322408676147,
  'random': 0.9890338778174235}}

In [115]:
classifier = ee.Classifier.smileRandomForest(100).train(training, label, bands)

In [116]:
# Classify the image with the same bands used for training.
result = image.select(bands).classify(classifier)

# # Display the clusters with random colors.
Map.addLayer(result.randomVisualizer(), {}, 'classfied')
Map.centerObject(result)
Map

### accuracy

In [111]:
train_accuracy = classifier.confusionMatrix()

In [112]:
train_accuracy.accuracy().getInfo()

0.9943820224719101

In [92]:
train_accuracy.kappa().getInfo()

0.9877752850144225

In [77]:
validated = validation.classify(classifier)

In [ ]:
validated.first().getInfo()

In [37]:
type(validated)

ee.featurecollection.FeatureCollection

In [121]:
test_accuracy= validated.errorMatrix('class', 'classification')

In [122]:
test_accuracy.getInfo()

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0],
 [0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 3, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 2, 0, 0, 0, 1, 0, 0, 0, 0, 0, 5, 0, 0],
 [0, 0, 1, 0, 0, 13, 0, 0, 0, 0, 0, 5, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 49, 0, 1],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 3, 0, 0]]

In [123]:
test_accuracy.accuracy().getInfo()

0.6568627450980392

In [124]:
test_accuracy.kappa().getInfo()

0.4307128049752831

In [125]:
test_accuracy.producersAccuracy().getInfo()

[[0],
 [0.5714285714285714],
 [0],
 [0],
 [0],
 [0.6842105263157895],
 [0],
 [0],
 [0.5],
 [0],
 [0],
 [0.9245283018867925],
 [0],
 [0]]

In [45]:
test_accuracy.consumersAccuracy().getInfo()

[[0, 0.75, 0, 0, 1, 0.5833333333333334, 0, 0, 1, 0, 0, 0.6875, 0, 0]]

In [46]:
import csv
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
training_csv = os.path.join(out_dir, 'train_accuracy.csv')
testing_csv = os.path.join(out_dir, 'test_accuracy.csv')

with open(training_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(train_accuracy.getInfo())

with open(testing_csv, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerows(test_accuracy.getInfo())

In [54]:
## export the result
geemap.ee_to_geopandas(validated)

,geometry,B1,B2,B3,B4,B5,B7,class,classification,random
0,None,0.129140,0.130680,0.147859,0.174170,0.188052,0.150880,2,5,0.977309
1,None,0.127752,0.116173,0.115758,0.147272,0.181981,0.156498,2,5,0.786695
2,None,0.143024,0.145188,0.160205,0.198080,0.210313,0.173352,2,5,0.969770
3,None,0.130528,0.124877,0.137982,0.177159,0.208289,0.159307,2,11,0.921350
4,None,0.130528,0.136483,0.145390,0.171182,0.212337,0.181779,2,5,0.994732
...,...,...,...,...,...,...,...,...,...,...
118,None,0.133305,0.133582,0.147859,0.180148,0.220431,0.181779,5,5,0.995008
119,None,0.141635,0.142286,0.155267,0.180148,0.210313,0.178970,5,5,0.976322
120,None,0.119421,0.107468,0.108351,0.120374,0.157697,0.142452,5,5,0.791350
121,None,0.134694,0.136483,0.142920,0.162216,0.206266,0.170543,5,2,0.993491


In [ ]:
landcover = result.set('classification_class_values', class_values)
landcover = landcover.set('classification_class_palette', class_palette)

In [39]:
import os

out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
out_file = os.path.join(out_dir, 'landcover.tif')

In [40]:
geemap.ee_export_image(result, filename=out_file, scale=900)

Generating URL ...
Please wait ...
Data downloaded to /Users/najah/Downloads/landcover.tif


In [56]:
type(image)

ee.image.Image

In [65]:
in_shp = './145044_20100428_roi/145044_20100428_200points.random_points/145044_20100428_200points.shp'
out_shp = './145044_20100428_roi/145044_20100428_200points.random_points/145044_20100428_200points_data'

In [62]:
in_fc = geemap.shp_to_ee(in_shp)

In [63]:
Map.addLayer(in_fc, {}, 'Cities')
Map

Map(bottom=33554732.0, center=[-85.0511287798066, 328.31335902214056], controls=(WidgetControl(options=['posit…

In [117]:
out_csv = ('./145044_20100428_models/RF/landsat.csv')
geemap.extract_values_to_points(in_fc, result, out_csv)


Generating URL ...
Please wait ...
Data downloaded to /Users/najah/work/internships/meghna/LT05_L1TP_145044_20100428_20161016_01_T1/145044_20100428_models/RF/landsat.csv


In [118]:
import pandas as pd
table = pd.read_csv(out_csv)
table['class'].value_counts()

11    71
10    68
5     21
1     15
13    10
4      9
2      3
8      2
12     1
Name: class, dtype: int64

In [119]:
cf = pd.crosstab(table['first'], table['class'])
cf

class,1,2,4,5,8,11,12,13
first,,,,,,,,
1.0,11,0,1,0,1,6,1,0
4.0,0,0,3,0,0,2,0,0
5.0,0,3,1,17,0,6,0,0
8.0,0,0,0,0,1,0,0,0
11.0,4,0,3,4,0,56,0,9
13.0,0,0,1,0,0,0,0,1


In [129]:
np.diag(cf).sum()

array([11,  0,  1,  0,  0,  0])

In [120]:
import numpy as np
np.diag(cf).sum() / cf.to_numpy().sum()

0.0916030534351145

In [96]:
cf / cf.sum(axis=1)

,1.0,2.0,4.0,5.0,8.0,11.0,12.0,13.0
first,,,,,,,,
1.0,0.60,0.0,0.000000,0.000000,0.5,0.081081,NaN,0.0
2.0,0.00,0.0,0.000000,0.041667,0.0,0.013514,NaN,0.0
4.0,0.00,0.0,0.714286,0.000000,0.0,0.027027,NaN,0.0
5.0,0.00,1.5,0.142857,0.708333,0.0,0.040541,NaN,0.0
8.0,0.00,0.0,0.000000,0.000000,0.5,0.013514,NaN,0.0
11.0,0.15,0.0,0.285714,0.125000,0.0,0.770270,NaN,4.5
13.0,0.00,0.0,0.142857,0.000000,0.0,0.000000,NaN,0.5
